In [ ]:
!pip install gspread PyDrive google-auth notion-client transformers
!pip install pdfplumber
!pip install python-pptx python-docx
!pip install PyPDF2
!pip install faiss-gpu
!pip install notion-client
!pip install PyDrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 635.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Retrieving

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from pptx import Presentation
from docx import Document
import io

# Inisialisasi Google Drive API
auth.authenticate_user()
service = build('drive', 'v3')

# Pilih Folder
folder_id = '1vC3Yq28hIiae-yYfpLBegbi38oebW9-D'  # Ganti dengan Folder ID Anda

# Query untuk mencari file dalam folder
query = f"'{folder_id}' in parents and (mimeType='application/vnd.google-apps.document' or mimeType='application/pdf' or mimeType='application/vnd.openxmlformats-officedocument.presentationml.presentation' or mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')"

# Mendapatkan daftar file di folder
results = service.files().list(q=query, pageSize=10, fields="nextPageToken, files(id, name, mimeType)").execute()
items = results.get('files', [])

if not items:
    print('Tidak ada file di dalam folder.')
else:
    print('Daftar file dalam folder:')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")

# Fungsi untuk mengunduh file dan mengekstrak teks
def streamfile(file_id, mime_type):
    request = service.files().get_media(fileId=file_id)
    buffer = io.BytesIO()
    downloader = MediaIoBaseDownload(buffer, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    buffer.seek(0)

    # Pengolahan berdasarkan tipe MIME
    if mime_type == 'application/vnd.google-apps.document':
        return buffer.getvalue().decode('utf-8')

    elif mime_type == 'application/pdf':
        from PyPDF2 import PdfReader
        pdf_reader = PdfReader(buffer)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        return text

    elif mime_type == 'application/vnd.openxmlformats-officedocument.presentationml.presentation':  # PPTX
        pptx_file = io.BytesIO(buffer.read())
        presentation = Presentation(pptx_file)
        text = ""
        for slide in presentation.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text += shape.text + "\n"
        return text

    elif mime_type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':  # DOCX
        docx_file = io.BytesIO(buffer.read())
        document = Document(docx_file)
        text = ""
        for paragraph in document.paragraphs:
            text += paragraph.text + "\n"
        return text

    return None

# Mengekstrak konten dari setiap file
files_content = []
for item in items:
    file_id = item['id']
    mime_type = item['mimeType']
    content = streamfile(file_id, mime_type)
    files_content.append((item['name'], content))

    print(f"Konten dari {item['name']}:\n{content[:100]}...\n")

Files in folder:
Paparan Keamanan Informasi (1).pdf (1ILuHcWnOE-Dr08Mg4Gd2bbyWXXE5wFnj) - application/pdf
pembinaan jf sandiman dan MI 2022 final.pptx (1OXQgz8hau-oyy9xuLsxR2973KP2gs-gJ) - application/vnd.openxmlformats-officedocument.presentationml.presentation
Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx (1JgNzHuvwOTLiFSTRFjyqYP1BDMTnA2v0) - application/vnd.openxmlformats-officedocument.wordprocessingml.document
Text from Paparan Keamanan Informasi (1).pdf:
Sosialisasi Keamanan
Informasi
Semarang, 21-24 Oktober 2024Dinas Komunikasi Informatika
Statistik da...

Text from pembinaan jf sandiman dan MI 2022 final.pptx:
”PEMBINAAN JF SANDIMAN DAN 
MANGGALA INFORMATIKA” 




















VISI INDONESIA 2045:
BERDAULA...

Text from Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx:
NOTULEN ACARA Enhancing Cybersecurity Awareness in Facing Cyber Threats

Hari 	: Kamis
Tanggal 	: 24...



# Cleaning

In [ ]:
import re
from nltk.tokenize import sent_tokenize
from multiprocessing import Pool
import logging

# Logging setup untuk melacak proses
logging.basicConfig(level=logging.INFO)

# Preprocessing teks
def preprocess_text(text):
    """
    Membersihkan teks dari spasi berlebih, karakter tidak penting,
    dan menormalkan teks untuk analisis lebih lanjut.
    """
    # Hilangkan karakter non-alfabet (menyisakan huruf, angka, dan tanda baca umum)
    text = re.sub(r'[^a-zA-Z0-9.,!?\'" \u00C0-\u017F]', '', text)
    # Hilangkan spasi ekstra
    text = re.sub(r'\s+', ' ', text)
    # Ubah ke huruf kecil untuk normalisasi
    text = text.lower().strip()
    return text

# Fungsi untuk memproses dokumen dan menghasilkan satu chunk besar
def preprocess_and_chunk(doc):
    """
    Membersihkan teks dokumen dan menghasilkan satu chunk besar dari seluruh teks.
    """
    name, text = doc
    logging.info(f"Memproses dokumen: {name}")
    text = preprocess_text(text)
    # Menghasilkan satu chunk yang mencakup seluruh teks yang sudah diproses
    return {"document": name, "content": text}

# Paralelisasi untuk mempercepat pemrosesan banyak dokumen
def process_documents(documents_text):
    """
    Memproses semua dokumen secara paralel.
    """
    with Pool() as pool:
        results = pool.map(preprocess_and_chunk, documents_text)
    return results

# Proses dokumen yang sudah diambil dari Google Drive
processed_data = process_documents(documents_text)

# Output hasil
for data in processed_data:
    print(data)

{'document': 'Paparan Keamanan Informasi (1).pdf', 'content': 'sosialisasi keamananinformasisemarang, 2124 oktober 2024dinas komunikasi informatikastatistik dan persandian kotasemarangperkembangan teknologi saat ini yangmeliputi big data dan artifisial inteligencememberikan dampak luar biasa bagi umatmanusia. saat ini data menjadi new goldyang perlu mendapat perhatian bagaimanapemanfaatannya, risikonya, sertakeamanannya.perubahan paradigma tersebut membuatkeamanan informasi menjadi perhatian utamadi era digital ini, di mana teknologi daninternet memainkan peran penting dalamkehidupan seharihari. perkembangan pesatteknologi informasi berbasis big data danartificial inteligence telah membawaperubahan signifikan terhadap cara informasidihasilkan, disimpan, dan disebarkan.keamanan informasi menjadiaspek penting yang harusdiperhatikan oleh organisasimaupun individu. tanpalangkahlangkah keamananyang memadai, risiko terjadinyakebocoran data, pencurianidentitas, sabotase, danserangan siber lai

#Embedding

In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
import os
import hashlib
from pinecone import Pinecone, ServerlessSpec
import numpy as np
from sentence_transformers import SentenceTransformer

# Inisialisasi Pinecone dengan API key
api_key = 'pcsk_7KJdHU_KaVtCztYZxvbuvmEcmWYjAJeiPPfL5VoYVdcF6kqethHkvUyzRaa8A4NKZVT8zE'  # Gantilah dengan API key Anda
pc = Pinecone(api_key=api_key)

# Inisialisasi Index
index_name = 'prototype'
dimension = 768  # Dimensi Embedding (Sesuaikan dengan model sentence transformernya)

# Buat Index jika tidak ada
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')  # Specify region here
    )

# Koneksi ke Indeks
index = pc.Index(index_name)

# Load Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Fungsi untuk membuat ID unik berdasarkan konten
def generate_id_from_content(content):
    return hashlib.sha256(content.encode('utf-8')).hexdigest()

# Generate embeddings untuk setiap chunk
chunk_texts = [item['content'] for item in processed_data]
embeddings = model.encode(chunk_texts)

# Convert embeddings menjadi numpy array
embeddings = np.array(embeddings)

# Menyimpan embeddings dalam Pinecone
# Setiap vektor memiliki ID unik berdasarkan hash konten
vectors = [
    (generate_id_from_content(processed_data[i]['content']), embeddings[i].tolist(), {
        "content": processed_data[i]['content'],
        "source": f"https://drive.google.com/file/d/{processed_data[i]['id']}/view"
    })
    for i in range(len(processed_data))
]

# Menambahkan embeddings ke Pinecone
index.upsert(vectors)

# Menyimpan mapping chunk untuk referensi di masa mendatang
chunk_mapping = {i: processed_data[i] for i in range(len(processed_data))}

print("Embeddings disimpan dalam Pinecone dengan metadata dan siap untuk pencarian!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings disimpan dalam Pinecone dengan metadata dan siap untuk pencarian!


In [ ]:
from notion_client import Client

# Inisialisasi notion token
notion = Client(auth="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")  # Gantilah dengan token API kamu

# Fungsi untuk mengambil data dari halaman Notion
def get_notion_page_data(page_id):
    page = notion.pages.retrieve(page_id)
    return page

# Contoh penggunaan: Ambil data dari halaman dengan ID yang sesuai
page_id = "176f7664c91b8084a529f816b6c0c559"  # Gantilah dengan ID halaman kamu
data = get_notion_page_data(page_id)
print(data)


{'object': 'page', 'id': '176f7664-c91b-8084-a529-f816b6c0c559', 'created_time': '2025-01-09T09:58:00.000Z', 'last_edited_time': '2025-01-09T14:45:00.000Z', 'created_by': {'object': 'user', 'id': '162d872b-594c-8126-9e70-0002d827a906'}, 'last_edited_by': {'object': 'user', 'id': '162d872b-594c-8126-9e70-0002d827a906'}, 'cover': None, 'icon': None, 'parent': {'type': 'page_id', 'page_id': '162f7664-c91b-8055-9c6e-d8604e39bc13'}, 'archived': False, 'in_trash': False, 'properties': {'title': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Biografi', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Biografi', 'href': None}]}}, 'url': 'https://www.notion.so/Biografi-176f7664c91b8084a529f816b6c0c559', 'public_url': None, 'request_id': 'e02005a3-3372-419f-a9f6-9950272f2d84'}


In [ ]:
from notion_client import Client

# Inisialisasi client dengan token API Notion
notion = Client(auth="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")  # Gantilah dengan token API kamu

# Fungsi untuk mengambil blok dari halaman Notion
def get_notion_page_text(page_id):
    blocks = notion.blocks.children.list(page_id)
    text_content = []

    for block in blocks['results']:
        if block['type'] == 'paragraph':  # Cek jika jenis blok adalah paragraf
            # Ambil isi teks dari rich_text
            paragraph_text = ''.join([text['text']['content'] for text in block['paragraph']['rich_text']])
            text_content.append(paragraph_text)

    return text_content

# Fungsi untuk mengambil data halaman dan URL Notion
def get_notion_page_data(page_id):
    page = notion.pages.retrieve(page_id)
    return page

# Ambil data halaman Notion
page_id = "176f7664-c91b-8084-a529-f816b6c0c559"  # Gantilah dengan ID halaman kamu
page_data = get_notion_page_data(page_id)

# Tampilkan teks dan URL halaman
text_data = get_notion_page_text(page_id)
for text in text_data:
    print(text)

# Ambil URL halaman dari data halaman
page_url = page_data['url']
print("URL:", page_url)

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed non risus. Suspendisse lectus tortor, dignissim sit amet, adipiscing nec, ultricies sed, dolor. Cras elementum ultrices diam. Maecenas ligula massa, varius a, semper congue, euismod non, mi. Proin porttitor, orci nec nonummy molestie, enim est eleifend mi, non fermentum diam nisl sit amet erat. Duis semper. Duis arcu massa, scelerisque vitae, consequat in, pretium a, enim. Pellentesque congue. Ut in risus volutpat libero pharetra tincidunt. Cras dapibus. Vivamus elementum semper nisi. Aenean vulputate eleifend tellus. Aenean leo ligula, porttitor eu, consequat vitae, eleifend ac, enim. Aliquam lorem ante, dapibus in, viverra quis, feugiat a, tellus. Phasellus viverra nulla ut metus varius laoreet. Quisque rutrum. Aenean imperdiet. Etiam ultricies nisi vel augue. Curabitur ullamcorper ultricies nisi. Nam eget dui. Etiam rhoncus. Maecenas tempus, tellus eget condimentum rhoncus, sem quam semper libero, sit amet adipiscing sem n

In [ ]:
from notion_client import Client
from sentence_transformers import SentenceTransformer
import numpy as np
from pinecone import Pinecone, ServerlessSpec

# Inisialisasi Pinecone dan model SentenceTransformer
api_key = 'pcsk_7KJdHU_KaVtCztYZxvbuvmEcmWYjAJeiPPfL5VoYVdcF6kqethHkvUyzRaa8A4NKZVT8zE'
pc = Pinecone(api_key=api_key)

# Koneksi ke index Pinecone
index_name = 'prototype'
index = pc.Index(index_name)
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Inisialisasi client Notion
notion = Client(auth="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")

# Fungsi untuk mengambil teks dari halaman Notion
def get_notion_page_text(page_id):
    blocks = notion.blocks.children.list(page_id)
    text_content = []
    for block in blocks['results']:
        if block['type'] == 'paragraph':
            paragraph_text = ''.join([text['text']['content'] for text in block['paragraph']['rich_text']])
            text_content.append(paragraph_text)
    return text_content

# Fungsi untuk mengambil data halaman dan URL Notion
def get_notion_page_data(page_id):
    page = notion.pages.retrieve(page_id)
    return page

# Ambil data halaman Notion
page_id = "176f7664-c91b-8084-a529-f816b6c0c559"
page_data = get_notion_page_data(page_id)

# Ambil data teks dari halaman Notion
notion_text_data = get_notion_page_text(page_id)

# Menghasilkan embeddings dari teks Notion
embeddings = model.encode(notion_text_data)

# Menyiapkan data untuk dimasukkan ke dalam Pinecone
vectors = []
for i, text in enumerate(notion_text_data):
    # ID unik, bisa diambil berdasarkan urutan atau ID tertentu
    vector_id = f"notion_{page_id}_{i}"  # Contoh ID berbasis halaman dan urutan
    metadata = {
        "content": text,
        "source": page_data['url']  # Menyertakan URL halaman Notion sebagai sumber
    }
    # Masukkan ke dalam list vektor
    vectors.append((vector_id, embeddings[i].tolist(), metadata))

# Menambahkan embeddings baru ke Pinecone
index.upsert(vectors)

print("Embeddings dari Notion berhasil dimasukkan ke Pinecone tanpa menimpa ID unik!")

Embeddings dari Notion berhasil dimasukkan ke Pinecone tanpa menimpa ID unik!


# query

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from pinecone import Pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

# Load the Hugging Face token from an environment variable
hf_token = os.getenv("HF_API_TOKEN")
if not hf_token:
    raise ValueError("Hugging Face API token not found. Please set 'HF_API_TOKEN' as an environment variable.")

# Log in using the token
login(hf_token)

# Fungsi untuk mendapatkan embedding query
def get_query_embedding(query):
    return model_sentence_bert.encode([query])

# Fungsi untuk mencari chunk relevan
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = get_query_embedding(query)  # Mendapatkan embedding query
    query_embedding = np.array(query_embedding).astype('float32')  # Menyusun embedding query ke format yang sesuai

    # Melakukan pencarian di Pinecone
    results = index.query(vector=query_embedding.tolist(), top_k=top_k, include_metadata=True)  # Query ke Pinecone

    # Mengambil hasil yang relevan dari metadata dengan validasi
    relevant_chunks = []
    for result in results['matches']:  # Mengiterasi hasil pencarian dari Pinecone
        metadata = result.get('metadata', {})  # Mendapatkan metadata dari hasil pencarian
        content = metadata.get('content', "Konten tidak tersedia.")  # Menyimpan konten jika ada
        source = metadata.get('source', "Sumber tidak diketahui.")  # Menyimpan sumber informasi
        relevant_chunks.append({"content": content, "source": source})  # Menambahkan ke list

    return relevant_chunks[:3]  # Batasi hanya 3 hasil teratas

# Muat model dan tokenizer Gemma-2-2B
model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    low_cpu_mem_usage=True
)

# Fungsi utama untuk menangani pertanyaan
def handle_question():
    query = input("Masukkan pertanyaan: ")  # Menerima input pertanyaan dari pengguna
    relevant_chunks = retrieve_relevant_chunks(query)  # Mendapatkan chunk relevan

    # Gabungkan chunk relevan menjadi konteks untuk input ke model
    context = " ".join([chunk['content'] for chunk in relevant_chunks])[:1000]  # Batasi panjang konteks
    input_text = f"Konteks: {context}\n\nPertanyaan: {query}\nJawaban: "

    # Tokenize input text untuk model Gemma
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    # Pindahkan inputs ke GPU
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Menghasilkan jawaban menggunakan model
    outputs = model.generate(
        **inputs,
        do_sample=True,
        max_new_tokens=150,  # Membatasi hanya jumlah token yang akan dihasilkan
        temperature=0.7,
        top_k=50,
        top_p=0.9
    )

    # Decode jawaban
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tambahkan sumber informasi ke jawaban (hanya tampilkan sumber relevan)
    if relevant_chunks:
        sources = f"Sumber informasi: {relevant_chunks[0]['source']}"  # Menampilkan sumber yang pertama
    else:
        sources = "Sumber informasi tidak ditemukan."

    print(f"Respon: {response}\n\n{sources}")


# Jalankan fungsi untuk menanggapi pertanyaan
handle_question()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Masukkan pertanyaan: lorem ipsum
Respon: Konteks: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed non risus. Suspendisse lectus tortor, dignissim sit amet, adipiscing nec, ultricies sed, dolor. Cras elementum ultrices diam. Maecenas ligula massa, varius a, semper congue, euismod non, mi. Proin porttitor, orci nec nonummy molestie, enim est eleifend mi, non fermentum diam nisl sit amet erat. Duis semper. Duis arcu massa, scelerisque vitae, consequat in, pretium a, enim. Pellentesque congue. Ut in risus volutpat libero pharetra tincidunt. Cras dapibus. Vivamus elementum semper nisi. Aenean vulputate eleifend tellus. Aenean leo ligula, porttitor eu, consequat vitae, eleifend ac, enim. Aliquam lorem ante, dapibus in, viverra quis, feugiat a, tellus. Phasellus viverra nulla ut metus varius laoreet. Quisque rutrum. Aenean imperdiet. Etiam ultricies nisi vel augue. Curabitur ullamcorper ultricies nisi. Nam eget dui. Etiam rhoncus. Maecenas tempus, tellus eget condimentum rhoncus,